In [5]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import operator
import pandas as pd

# file = open()
X = []
y = []  # decoy, target
# Lines = file.readlines()
count = 0
# with open("/home/ip2/data/test.sqt") as fp:
# with open("/home/ip2/data/sqt/20220727_TypeAcomplex_Elastase_S2-A3_1_10495_nopd.sqt") as fp:
prev_sline =""
line = ""
# with open("/home/ip2/data/claricep_tsri/20220727_TypeAcomplex_Elastase_S2-A3_1_10495_nopd_shuffled.sqt") as fp:
# with open("/home/ip2/data/claricep_tsri/20220727_TypeAcomplex_Elastase_S2-A3_1_10495_nopd_reversed.sqt") as fp:
with open("/Users/claricepark/Discriminant-Analysis/Botox_debruijn.sqt") as fp:
    
    while True:
        prev_sline = line
        line = fp.readline()
        if not line:
            break

        if line.startswith("H\t"):
            continue

        sline = line.strip()
        count += 1
        # print(sline)
        # if(count>200):
        #     break


        decoy_label = 1
        if line.startswith("M\t1\t") and prev_sline.startswith("S\t"):

            # print("prev\t", prev_sline.strip())
            # print("1", line.strip())
            xcorr = line.split("\t")[5]
            # print(xcorr)

            lline = fp.readline()

            if (lline.split("\t")[1].startswith("Reverse")):
                decoy_label = 0
            # print (lline.strip())
            # print (decoy_label)
            # print ("l line\t", lline.split("\t")[1])
            # print(lline)
            while True:
                m2line = fp.readline().strip()
                # print("2", m2line)

                if (m2line.startswith("M\t2\t")):
                    break

            # m2line = fp.readline()
            # print ("m2  line\t", m2line.strip())
            deltacn = m2line.split("\t")[4]

            # print("delta cn\t", deltacn)
            
            if(decoy_label == '1' and float(xcorr)<2.5 and float(deltacn)<0.2):
                continue
            
            X.append([float(xcorr), float(deltacn)])
            y.append(int(decoy_label))

clf = LinearDiscriminantAnalysis()
clf.fit(X, y)
# print(clf.predict([[3.2, 0.001]]))
# print(clf.predict([[1.2, 0.0001]]))
# print(clf.predict([[1.05, 0.0001]]))
# print(clf.predict([[0.002, 0.00001]]))

# print(clf.predict_proba([[3.2, 0.001]]))
# print(clf.predict_proba([[1.2, 0.0001]]))
# print(clf.predict_proba([[1.05, 0.0001]]))
# print( type(clf.predict_proba([[0.002, 0.00001]])) )

# count = 0
pred_arr = []
for score, label in zip(X, y):
    # count += 1
    
    # if(count>2):
    #     break
    
    pred_scores = clf.predict_proba([score])
    # print (pred_scores)
    # print (type(pred_scores[0][1]))
    pred_arr.append( [pred_scores[0][1], score[0], score[1], label] )
    
# print (pred_arr)
df = pd.DataFrame(pred_arr, columns=['pred_value', 'xcorr', 'delta_cn', 'label'])

# df
df = df.sort_values(by=['pred_value'], ascending=True)
target_count = 0
decoy_count = 0

# print (df)
df.to_csv("/Users/claricepark/Discriminant-Analysis/test.csv")

count = 0
target_total = df[df['label']==1].shape[0]
decoy_total = df[df['label']==0].shape[0]
# decoy_count1 = df['label'==0].sum()
# print (target_count1, decoy_count1)

for index, row in df.iterrows():
    pred_value = row['pred_value']
    label = row['label']
    if(label == 1):
        target_count += 1
        target_total -= 1
    else:
        decoy_count += 1
        decoy_total -= 1
        
    # print (pred_value, row['xcorr'], row['delta_cn'], target_count, decoy_count, label)
    count += 1
    # if (count>2):
    #     break
    
    # print (count, target_count, decoy_count)
    
        
    # print(pred_value, label, target_count, decoy_count, decoy_count/target_count)
        
    # fdr = decoy_count/target_count
    fdr = decoy_total/target_total
    
    # if(fdr>0.01 and count>1000):
    if(fdr<=0.01):
        break
        
    
print ("target id:", target_total)
print ("decoy id:", decoy_total)
# print ("target_total id:", target_total)
# print ("decoy_total id:", decoy_total)
print ("fdr:", (decoy_count-1)/target_count)

target id: 2468
decoy id: 24
fdr: 0.746040182842564
